In [4]:
class Node:
    def __init__(self, key, color="RED"):
        self.key = key
        self.left = None
        self.right = None
        self.parent = None
        self.color = color  # "RED" or "BLACK"


class RedBlackTree:
    def __init__(self):
        self.NIL = Node(None, "BLACK")  # sentinel node
        self.root = self.NIL

    def search(self, key):
        """Search for a key in the tree"""
        return self._search_helper(self.root, key)

    def _search_helper(self, node, key):
        if node == self.NIL:
            return None
        if key == node.key:
            return node
        if key < node.key:
            return self._search_helper(node.left, key)
        return self._search_helper(node.right, key)

    def minimum(self, node):
        """Find the minimum key in the subtree rooted at node"""
        current = node
        while current.left != self.NIL:
            current = current.left
        return current

    def maximum(self, node):
        """Find the maximum key in the subtree rooted at node"""
        current = node
        while current.right != self.NIL:
            current = current.right
        return current

    def successor(self, x):
        """Find the successor of a node in the tree"""
        if x.right != self.NIL:
            return self.minimum(x.right)

        y = x.parent
        while y != self.NIL and x == y.right:
            x = y
            y = y.parent
        return y

    def predecessor(self, x):
        """Find the predecessor of a node in the tree"""
        if x.left != self.NIL:
            return self.maximum(x.left)

        y = x.parent
        while y != self.NIL and x == y.left:
            x = y
            y = y.parent
        return y

    def left_rotate(self, x):
        """Perform left rotation on node x"""
        y = x.right
        x.right = y.left

        if y.left != self.NIL:
            y.left.parent = x

        y.parent = x.parent

        if x.parent == self.NIL:
            self.root = y
        elif x == x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y

        y.left = x
        x.parent = y

    def right_rotate(self, x):
        """Perform right rotation on node x"""
        y = x.left
        x.left = y.right

        if y.right != self.NIL:
            y.right.parent = x

        y.parent = x.parent

        if x.parent == self.NIL:
            self.root = y
        elif x == x.parent.right:
            x.parent.right = y
        else:
            x.parent.left = y

        y.right = x
        x.parent = y

    def insert(self, key):
        """Insert a key into the tree"""
        node = Node(key)
        node.left = self.NIL
        node.right = self.NIL

        y = self.NIL
        x = self.root

        # Find the position to insert the new node
        while x != self.NIL:
            y = x
            if node.key < x.key:
                x = x.left
            else:
                x = x.right

        node.parent = y

        if y == self.NIL:
            self.root = node  # Tree was empty
        elif node.key < y.key:
            y.left = node
        else:
            y.right = node

        # Fix Red-Black properties
        self._insert_fixup(node)

        return node

    def _insert_fixup(self, k):
        """Fix Red-Black Tree properties after insertion"""
        while k != self.root and k.parent.color == "RED":
            if k.parent == k.parent.parent.left:  # k's parent is a left child
                y = k.parent.parent.right  # y is k's uncle

                if y.color == "RED":  # Case 1: Uncle is red
                    k.parent.color = "BLACK"
                    y.color = "BLACK"
                    k.parent.parent.color = "RED"
                    k = k.parent.parent
                else:
                    if k == k.parent.right:  # Case 2: Uncle is black and k is a right child
                        k = k.parent
                        self.left_rotate(k)

                    # Case 3: Uncle is black and k is a left child
                    k.parent.color = "BLACK"
                    k.parent.parent.color = "RED"
                    self.right_rotate(k.parent.parent)
            else:  # k's parent is a right child
                y = k.parent.parent.left  # y is k's uncle

                if y.color == "RED":  # Case 1: Uncle is red
                    k.parent.color = "BLACK"
                    y.color = "BLACK"
                    k.parent.parent.color = "RED"
                    k = k.parent.parent
                else:
                    if k == k.parent.left:  # Case 2: Uncle is black and k is a left child
                        k = k.parent
                        self.right_rotate(k)

                    # Case 3: Uncle is black and k is a right child
                    k.parent.color = "BLACK"
                    k.parent.parent.color = "RED"
                    self.left_rotate(k.parent.parent)

        self.root.color = "BLACK"  # Ensure root is black

    def transplant(self, u, v):
        """Replace subtree rooted at u with subtree rooted at v"""
        if u.parent == self.NIL:
            self.root = v
        elif u == u.parent.left:
            u.parent.left = v
        else:
            u.parent.right = v
        v.parent = u.parent

    def delete(self, key):
        """Delete a node with the given key"""
        z = self._search_helper(self.root, key)
        if z is None or z == self.NIL:
            return False  # Key not found

        self._delete_node(z)
        return True

    def _delete_node(self, z):
        """Delete the given node and fix the Red-Black properties"""
        y = z
        y_original_color = y.color

        if z.left == self.NIL:
            x = z.right
            self.transplant(z, z.right)
        elif z.right == self.NIL:
            x = z.left
            self.transplant(z, z.left)
        else:
            y = self.minimum(z.right)
            y_original_color = y.color
            x = y.right

            if y.parent == z:
                x.parent = y
            else:
                self.transplant(y, y.right)
                y.right = z.right
                y.right.parent = y

            self.transplant(z, y)
            y.left = z.left
            y.left.parent = y
            y.color = z.color

        if y_original_color == "BLACK":
            self._delete_fixup(x)

    def _delete_fixup(self, x):
        """Fix Red-Black Tree properties after deletion"""
        while x != self.root and x.color == "BLACK":
            if x == x.parent.left:
                w = x.parent.right

                if w.color == "RED":  # Case 1: x's sibling is red
                    w.color = "BLACK"
                    x.parent.color = "RED"
                    self.left_rotate(x.parent)
                    w = x.parent.right

                if w.left.color == "BLACK" and w.right.color == "BLACK":  # Case 2: x's sibling has two black children
                    w.color = "RED"
                    x = x.parent
                else:
                    if w.right.color == "BLACK":  # Case 3: x's sibling has a red left child and black right child
                        w.left.color = "BLACK"
                        w.color = "RED"
                        self.right_rotate(w)
                        w = x.parent.right

                    # Case 4: x's sibling has a red right child
                    w.color = x.parent.color
                    x.parent.color = "BLACK"
                    w.right.color = "BLACK"
                    self.left_rotate(x.parent)
                    x = self.root
            else:
                w = x.parent.left

                if w.color == "RED":  # Case 1: x's sibling is red
                    w.color = "BLACK"
                    x.parent.color = "RED"
                    self.right_rotate(x.parent)
                    w = x.parent.left

                if w.right.color == "BLACK" and w.left.color == "BLACK":  # Case 2: x's sibling has two black children
                    w.color = "RED"
                    x = x.parent
                else:
                    if w.left.color == "BLACK":  # Case 3: x's sibling has a red right child and black left child
                        w.right.color = "BLACK"
                        w.color = "RED"
                        self.left_rotate(w)
                        w = x.parent.left

                    # Case 4: x's sibling has a red left child
                    w.color = x.parent.color
                    x.parent.color = "BLACK"
                    w.left.color = "BLACK"
                    self.right_rotate(x.parent)
                    x = self.root

        x.color = "BLACK"

    def inorder_traversal(self):
        """Return the inorder traversal of the tree"""
        result = []
        self._inorder_helper(self.root, result)
        return result

    def _inorder_helper(self, node, result):
        if node != self.NIL:
            self._inorder_helper(node.left, result)
            result.append(node.key)
            self._inorder_helper(node.right, result)

    def preorder_traversal(self):
        """Return the preorder traversal of the tree"""
        result = []
        self._preorder_helper(self.root, result)
        return result

    def _preorder_helper(self, node, result):
        if node != self.NIL:
            result.append(node.key)
            self._preorder_helper(node.left, result)
            self._preorder_helper(node.right, result)

    def check_properties(self):
        """Check if the tree satisfies Red-Black Tree properties"""
        if self.root == self.NIL:
            return True

        # Property 1: Every node is either red or black - enforced by implementation

        # Property 2: The root is black
        if self.root.color != "BLACK":
            print("Property violation: Root is not black")
            return False

        # Properties 3 & 4: Check for red nodes having only black children and black height
        black_height = -1

        def check_subtree(node, black_count):
            nonlocal black_height

            if node == self.NIL:
                # Property 3: All leaf nodes (NIL) are black - enforced by implementation
                if black_height == -1:
                    black_height = black_count
                    return True
                return black_count == black_height

            # Property 4: If a node is red, both its children are black
            if node.color == "RED" and (node.left.color == "RED" or node.right.color == "RED"):
                print(f"Property violation: Red node {node.key} has a red child")
                return False

            # Count black nodes on path
            next_count = black_count + (1 if node.color == "BLACK" else 0)

            # Check both subtrees
            return (check_subtree(node.left, next_count) and
                    check_subtree(node.right, next_count))

        return check_subtree(self.root, 0)

    def __contains__(self, key):
        """Allow 'in' operator: key in tree"""
        return self.search(key) is not None

    def height(self):
        """Return the height of the tree"""
        return self._height_helper(self.root)

    def _height_helper(self, node):
        if node == self.NIL:
            return -1

        left_height = self._height_helper(node.left)
        right_height = self._height_helper(node.right)

        return max(left_height, right_height) + 1

    def print_tree(self):
        """Print a simple representation of the tree structure with colors"""
        if self.root == self.NIL:
            print("Empty tree")
            return

        def get_tree_lines(node, level=0):
            if node == self.NIL:
                return ["NIL(B)"], 1, 1

            color = "R" if node.color == "RED" else "B"
            node_str = f"{node.key}({color})"

            # Handle leaf case
            if node.left == self.NIL and node.right == self.NIL:
                return [node_str], len(node_str), 1

            # Get lines for left and right subtrees
            left_lines, left_pos, left_width = get_tree_lines(node.left, level + 1)
            right_lines, right_pos, right_width = get_tree_lines(node.right, level + 1)

            # Calculate current node position and width
            middle = max(right_pos + left_width - left_pos, len(node_str), 2)
            pos = left_pos + middle // 2
            width = left_pos + middle + right_width - right_pos

            # Adjust for minimum width
            while len(node_str) < middle:
                node_str = " " + node_str + " "
                if len(node_str) >= middle:
                    break
                node_str = node_str[:-1]

            # Add connecting lines
            connections = " " * (left_pos + middle // 2 - 1)
            if left_pos + middle // 2 > 0:
                connections += "/"
            connections += " " * (right_pos - left_pos - middle // 2)
            if right_pos > left_pos + middle // 2:
                connections += "\\"

            # Combine lines from left and right subtrees
            lines = [node_str]
            lines.append(connections)

            # Add subtree lines with appropriate padding
            for i in range(max(len(left_lines), len(right_lines))):
                left_line = left_lines[i] if i < len(left_lines) else " " * left_width
                right_line = right_lines[i] if i < len(right_lines) else " " * right_width
                lines.append(left_line + " " * (width - left_width - right_width) + right_line)

            return lines, pos, width

        lines, _, _ = get_tree_lines(self.root)
        for line in lines:
            print(line)


# Output scenario to demonstrate the Red-Black Tree
def output_scenario():
    print("\n===== Red-Black Tree Output Scenario =====\n")

    # Create a new Red-Black Tree
    tree = RedBlackTree()
    print("Creating a new Red-Black Tree")

    # Insert some values
    print("\nInserting values: 10, 20, 30, 15, 25, 5, 35")
    for value in [10, 20, 30, 15, 25, 5, 35]:
        tree.insert(value)
        print(f"Inserted {value}")

    # Print the tree structure
    print("\nTree structure after insertions:")
    tree.print_tree()

    # Check if the tree maintains Red-Black properties
    print("\nChecking Red-Black properties:", "Valid" if tree.check_properties() else "Invalid")

    # Print traversals
    print("\nInorder traversal:", tree.inorder_traversal())
    print("Preorder traversal:", tree.preorder_traversal())

    # Search for values
    print("\nSearching for values:")
    for value in [15, 40]:
        result = "Found" if tree.search(value) else "Not found"
        print(f"Search for {value}: {result}")

    # Delete some values
    print("\nDeleting values: 20, 5")
    for value in [20, 5]:
        if tree.delete(value):
            print(f"Deleted {value}")
        else:
            print(f"Value {value} not found")

    # Print the tree after deletion
    print("\nTree structure after deletions:")
    tree.print_tree()

    # Verify Red-Black properties are still maintained
    print("\nChecking Red-Black properties after deletion:",
          "Valid" if tree.check_properties() else "Invalid")

    # Final traversal
    print("\nFinal inorder traversal:", tree.inorder_traversal())

    # Tree height
    print(f"Tree height: {tree.height()}")

    print("\n===== End of Output Scenario =====")


# Test the Red-Black Tree implementation
def test_red_black_tree():
    tree = RedBlackTree()

    # Test insertion
    test_data = [7, 3, 18, 10, 22, 8, 11, 26, 2, 6, 13]
    for num in test_data:
        tree.insert(num)

    assert tree.check_properties() == True, "Red-Black properties violated after insertion"
    assert tree.inorder_traversal() == sorted(test_data), "Inorder traversal should return sorted elements"

    # Test search
    for num in test_data:
        assert tree.search(num) is not None, f"Search failed for {num}"
    assert tree.search(100) is None, "Search should return None for non-existent elements"

    # Test contains operator
    for num in test_data:
        assert num in tree, f"Contains operator failed for {num}"
    assert 100 not in tree, "Contains operator should return False for non-existent elements"

    # Test deletion
    tree.delete(18)
    assert 18 not in tree, "18 should be removed from the tree"
    assert tree.check_properties() == True, "Red-Black properties violated after deletion"

    # Test multiple deletions
    for num in [7, 22, 8, 2]:
        tree.delete(num)
        assert num not in tree, f"{num} should be removed from the tree"
        assert tree.check_properties() == True, f"Red-Black properties violated after deleting {num}"

    # Verify remaining elements
    remaining = set(test_data) - {7, 18, 22, 8, 2}
    for num in remaining:
        assert num in tree, f"{num} should still be in the tree"

    # Test edge cases
    edge_tree = RedBlackTree()

    # Insert and delete a single node
    edge_tree.insert(5)
    assert edge_tree.check_properties() == True, "Red-Black properties violated with single node"
    edge_tree.delete(5)
    assert 5 not in edge_tree, "Tree should be empty after deleting the only node"
    assert edge_tree.root == edge_tree.NIL, "Root should be NIL after deleting the only node"

    # Test large number of insertions and deletions
    large_tree = RedBlackTree()
    large_data = list(range(1, 101))
    for num in large_data:
        large_tree.insert(num)

    assert large_tree.check_properties() == True, "Red-Black properties violated with large dataset"

    # Delete half of the elements
    for num in large_data[:50]:
        large_tree.delete(num)
        assert num not in large_tree, f"{num} should be removed from the tree"

    assert large_tree.check_properties() == True, "Red-Black properties violated after multiple deletions"

    print("All tests passed!")


# Run both the tests and output scenario
if __name__ == "__main__":
    test_red_black_tree()
    output_scenario()

All tests passed!

===== Red-Black Tree Output Scenario =====

Creating a new Red-Black Tree

Inserting values: 10, 20, 30, 15, 25, 5, 35
Inserted 10
Inserted 20
Inserted 30
Inserted 15
Inserted 25
Inserted 5
Inserted 35

Tree structure after insertions:
 20(B) 
        /
10(B)30(B)
     /      /
5(R)   15(R)25(R)    35(R)

Checking Red-Black properties: Valid

Inorder traversal: [5, 10, 15, 20, 25, 30, 35]
Preorder traversal: [20, 10, 5, 15, 30, 25, 35]

Searching for values:
Search for 15: Found
Search for 40: Not found

Deleting values: 20, 5
Deleted 20
Deleted 5

Tree structure after deletions:
25(B)
    /
10(B)   30(B)
  /  \     /  \
NIL(B)15(R)   NIL(B)35(R)

Checking Red-Black properties after deletion: Valid

Final inorder traversal: [10, 15, 25, 30, 35]
Tree height: 2

===== End of Output Scenario =====
